# Lab 2.2: Variation & Heritability Analyzer
## Unit 2: Evolution & Extinction

### 🎯 Learning Objectives
- Distinguish heritable vs non-heritable variation
- Calculate heritability coefficients (h²)
- Apply breeder's equation to predict selection response
- Partition variance into genetic and environmental components
- Understand realized heritability

### 📖 Connection to Course
Covers **Sources of Variation** from Unit 2: The raw material for evolution

### 🧬 The Big Question
**How much variation is heritable?** Without heritable variation, no evolution!

In [ ]:
# === GOOGLE COLAB SETUP ===
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("✓ Widgets enabled")
except:
    print("✓ Running outside Colab")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import *
from IPython.display import display, HTML
from datetime import datetime
from scipy import stats

np.random.seed(42)
print("✓ Libraries loaded!")

## Part 1: Heritability Theory

### What is Heritability?

**h² = Proportion of phenotypic variance due to genetic variance**

### Key Equations

**Phenotypic Variance:**
## V_P = V_G + V_E + V_GE

Where:
- V_P = Total phenotypic variance
- V_G = Genetic variance
- V_E = Environmental variance
- V_GE = Gene-environment interaction

**Narrow-sense Heritability:**
## h² = V_A / V_P

Where V_A = Additive genetic variance (the portion that responds to selection)

**Broad-sense Heritability:**
## H² = V_G / V_P

Includes all genetic variance (additive, dominance, epistasis)

### Breeder's Equation

**Predicts response to selection:**
## R = h² × S

Where:
- R = Response (change in offspring mean)
- h² = Narrow-sense heritability
- S = Selection differential (difference between selected parents and population mean)

### Parent-Offspring Regression

**Estimate h² from regression:**
## h² = slope of mid-parent vs offspring regression

**Why?**
- Offspring inherit half genes from each parent
- Mid-parent value = average of both parents
- Slope tells how much offspring resemble parents
- Perfect heritability (h²=1): slope = 1
- No heritability (h²=0): slope = 0

### Interpretation

**h² = 0:** No genetic influence (all environmental)  
**h² = 0.3:** Low heritability (30% genetic)  
**h² = 0.5:** Moderate heritability (50% genetic)  
**h² = 0.8:** High heritability (80% genetic)  
**h² = 1.0:** Complete heritability (100% genetic)  

**CRITICAL:** h² is population-specific and environment-specific!

## Part 2: Example Trait Database

In [ ]:
# Real heritability estimates for various traits
trait_heritabilities = {
    'Human height': {'h2': 0.80, 'category': 'Human morphology', 'unit': 'cm'},
    'Human IQ': {'h2': 0.50, 'category': 'Human cognitive', 'unit': 'points'},
    'Human BMI': {'h2': 0.47, 'category': 'Human morphology', 'unit': 'kg/m²'},
    'Cow milk yield': {'h2': 0.35, 'category': 'Agriculture', 'unit': 'liters'},
    'Chicken egg count': {'h2': 0.20, 'category': 'Agriculture', 'unit': 'eggs'},
    'Corn yield': {'h2': 0.70, 'category': 'Agriculture', 'unit': 'bushels'},
    'Dog body size': {'h2': 0.90, 'category': 'Animal', 'unit': 'kg'},
    'Bird beak depth': {'h2': 0.79, 'category': 'Wild animal', 'unit': 'mm'},
    'Butterfly wing pattern': {'h2': 0.85, 'category': 'Wild animal', 'unit': 'score'},
    'Flower color': {'h2': 0.95, 'category': 'Plant', 'unit': 'wavelength'},
    'Seed size': {'h2': 0.65, 'category': 'Plant', 'unit': 'mg'},
    'Schizophrenia risk': {'h2': 0.64, 'category': 'Human disease', 'unit': 'risk'},
    'Type 2 diabetes risk': {'h2': 0.26, 'category': 'Human disease', 'unit': 'risk'}
}

# Visualize
traits = list(trait_heritabilities.keys())
h2_values = [data['h2'] for data in trait_heritabilities.values()]
colors_map = {
    'Human morphology': '#E74C3C',
    'Human cognitive': '#9B59B6',
    'Human disease': '#34495E',
    'Agriculture': '#F39C12',
    'Animal': '#3498DB',
    'Wild animal': '#1ABC9C',
    'Plant': '#2ECC71'
}
bar_colors = [colors_map[trait_heritabilities[t]['category']] for t in traits]

fig = go.Figure(go.Bar(
    y=traits,
    x=h2_values,
    orientation='h',
    marker_color=bar_colors,
    text=[f"{h:.2f}" for h in h2_values],
    textposition='outside'
))

fig.update_layout(
    title='<b>Heritability Estimates (h²) for Various Traits</b>',
    xaxis_title='Heritability (h²)',
    xaxis=dict(range=[0, 1]),
    height=600
)

fig.show()

print("\nHERITABILITY DATABASE")
print("="*70)
print(f"{'Trait':<30}{'h²':<10}{'Category'}")
print("="*70)
for trait, data in trait_heritabilities.items():
    print(f"{trait:<30}{data['h2']:<10}{data['category']}")
print("\n✓ Database ready!")

## Part 3: Heritability Calculator

In [ ]:
def calculate_heritability(V_G, V_E):
    """
    Calculate heritability from variance components
    """
    V_P = V_G + V_E
    h2 = V_G / V_P if V_P > 0 else 0
    
    # Visualization
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Variance Components', 'Heritability'),
        specs=[[{'type': 'bar'}, {'type': 'indicator'}]]
    )
    
    # 1. Variance pie
    fig.add_trace(go.Bar(
        x=['Genetic (V_G)', 'Environmental (V_E)', 'Total (V_P)'],
        y=[V_G, V_E, V_P],
        marker_color=['#E74C3C', '#3498DB', '#2ECC71'],
        text=[f"{V_G:.2f}", f"{V_E:.2f}", f"{V_P:.2f}"],
        textposition='outside',
        showlegend=False
    ), row=1, col=1)
    
    # 2. h² gauge
    fig.add_trace(go.Indicator(
        mode="gauge+number",
        value=h2,
        title={'text': "h²"},
        gauge={
            'axis': {'range': [0, 1]},
            'bar': {'color': '#E74C3C'},
            'steps': [
                {'range': [0, 0.3], 'color': 'lightgray'},
                {'range': [0.3, 0.5], 'color': '#F39C12'},
                {'range': [0.5, 0.8], 'color': '#E67E22'},
                {'range': [0.8, 1], 'color': '#C0392B'}],
            'threshold': {
                'line': {'color': 'red', 'width': 4},
                'thickness': 0.75,
                'value': 0.5}
        }
    ), row=1, col=2)
    
    fig.update_yaxes(title_text="Variance", row=1, col=1)
    fig.update_layout(height=400, title_text='<b>Heritability Calculation</b>')
    
    # Results
    print("\n" + "="*70)
    print("HERITABILITY CALCULATION")
    print("="*70)
    print(f"\nVARIANCE COMPONENTS:")
    print(f"  Genetic variance (V_G):       {V_G:.2f}")
    print(f"  Environmental variance (V_E): {V_E:.2f}")
    print(f"  Total variance (V_P):         {V_P:.2f}")
    print(f"\nCALCULATION:")
    print(f"  h² = V_G / V_P")
    print(f"     = {V_G:.2f} / {V_P:.2f}")
    print(f"     = {h2:.3f}")
    print(f"\nHERITABILITY: h² = {h2:.3f} ({h2*100:.1f}%)")
    
    # Interpretation
    if h2 < 0.3:
        print(f"\nINTERPRETATION: LOW heritability")
        print(f"  → Environment dominates")
        print(f"  → Selection will be SLOW")
        print(f"  → Hard to breed for this trait")
    elif h2 < 0.5:
        print(f"\nINTERPRETATION: MODERATE heritability")
        print(f"  → Both genes and environment matter")
        print(f"  → Selection moderately effective")
    elif h2 < 0.8:
        print(f"\nINTERPRETATION: HIGH heritability")
        print(f"  → Genetics dominate")
        print(f"  → Selection will be EFFECTIVE")
        print(f"  → Easy to breed for this trait")
    else:
        print(f"\nINTERPRETATION: VERY HIGH heritability")
        print(f"  → Almost entirely genetic")
        print(f"  → Selection very effective")
        print(f"  → Mendelian traits (e.g., flower color)")
    
    print("="*70)
    
    fig.show()

# Interactive
VG_slider = FloatSlider(value=40, min=0, max=100, step=5, description='V_G:')
VE_slider = FloatSlider(value=60, min=0, max=100, step=5, description='V_E:')

display(HTML("<h3>🧬 Heritability Calculator</h3>"))
interact(calculate_heritability, V_G=VG_slider, V_E=VE_slider);

## Part 4: Breeder's Equation Simulator

In [ ]:
def breeders_equation(h2, pop_mean, selected_mean):
    """
    Predict selection response using breeder's equation
    """
    # Calculate selection differential
    S = selected_mean - pop_mean
    
    # Calculate response
    R = h2 * S
    
    # Offspring mean
    offspring_mean = pop_mean + R
    
    # Visualization
    fig = go.Figure()
    
    # Population distribution
    x_vals = np.linspace(pop_mean - 30, pop_mean + 30, 100)
    y_pop = stats.norm.pdf(x_vals, pop_mean, 10)
    
    fig.add_trace(go.Scatter(
        x=x_vals, y=y_pop,
        fill='tozeroy',
        fillcolor='rgba(52, 152, 219, 0.3)',
        line=dict(color='#3498DB', width=2),
        name='Population'
    ))
    
    # Selected parents distribution
    y_selected = stats.norm.pdf(x_vals, selected_mean, 8)
    fig.add_trace(go.Scatter(
        x=x_vals, y=y_selected,
        fill='tozeroy',
        fillcolor='rgba(231, 76, 60, 0.3)',
        line=dict(color='#E74C3C', width=2),
        name='Selected Parents'
    ))
    
    # Offspring distribution
    y_offspring = stats.norm.pdf(x_vals, offspring_mean, 10)
    fig.add_trace(go.Scatter(
        x=x_vals, y=y_offspring,
        fill='tozeroy',
        fillcolor='rgba(46, 204, 113, 0.3)',
        line=dict(color='#2ECC71', width=2, dash='dash'),
        name='Offspring'
    ))
    
    # Mark means
    fig.add_vline(x=pop_mean, line_dash="dash", 
                 annotation_text="Pop mean", line_color='#3498DB')
    fig.add_vline(x=selected_mean, line_dash="dash",
                 annotation_text="Selected mean", line_color='#E74C3C')
    fig.add_vline(x=offspring_mean, line_dash="dash",
                 annotation_text="Offspring mean", line_color='#2ECC71')
    
    # Add arrows for S and R
    fig.add_annotation(
        x=(pop_mean + selected_mean)/2, y=max(y_pop)*1.1,
        text=f"S = {S:.1f}",
        showarrow=True, arrowhead=2, arrowcolor='#E74C3C',
        ax=40, ay=-40
    )
    fig.add_annotation(
        x=(pop_mean + offspring_mean)/2, y=max(y_pop)*0.9,
        text=f"R = {R:.1f}",
        showarrow=True, arrowhead=2, arrowcolor='#2ECC71',
        ax=-40, ay=40
    )
    
    fig.update_layout(
        title='<b>Breeder\'s Equation: R = h² × S</b>',
        xaxis_title='Trait Value',
        yaxis_title='Frequency',
        height=500
    )
    
    # Results
    print("\n" + "="*70)
    print("BREEDER'S EQUATION")
    print("="*70)
    print(f"\nGIVEN:")
    print(f"  Heritability (h²):           {h2:.2f}")
    print(f"  Population mean:             {pop_mean:.1f}")
    print(f"  Selected parents mean:       {selected_mean:.1f}")
    print(f"\nCALCULATION:")
    print(f"  Selection differential (S) = {selected_mean:.1f} - {pop_mean:.1f} = {S:.1f}")
    print(f"\n  Response (R) = h² × S")
    print(f"               = {h2:.2f} × {S:.1f}")
    print(f"               = {R:.1f}")
    print(f"\n  Offspring mean = Population mean + R")
    print(f"                 = {pop_mean:.1f} + {R:.1f}")
    print(f"                 = {offspring_mean:.1f}")
    print(f"\nRESULT:")
    print(f"  Offspring mean: {offspring_mean:.1f}")
    print(f"  Change from population: {R:.1f} ({R/pop_mean*100:.1f}%)")
    print(f"  Realized h²: {R/S:.3f}" if S != 0 else "")
    print(f"\nKEY INSIGHT:")
    if h2 > 0.8:
        print(f"  High h² → Offspring close to selected parents")
        print(f"  Selection very effective!")
    elif h2 > 0.5:
        print(f"  Moderate h² → Offspring intermediate")
        print(f"  Selection moderately effective")
    else:
        print(f"  Low h² → Offspring close to population mean")
        print(f"  Selection not very effective (environment dominates)")
    print("="*70)
    
    fig.show()

# Interactive
h2_slider = FloatSlider(value=0.6, min=0, max=1, step=0.05, description='h²:')
pop_slider = FloatSlider(value=100, min=50, max=150, step=5, description='Pop mean:')
sel_slider = FloatSlider(value=120, min=50, max=150, step=5, description='Selected mean:')

display(HTML("<h3>🌾 Breeder's Equation Simulator</h3>"))
interact(breeders_equation, h2=h2_slider, pop_mean=pop_slider, selected_mean=sel_slider);

## Part 5: Parent-Offspring Regression

In [ ]:
def parent_offspring_regression(h2_true):
    """
    Simulate parent-offspring regression to estimate h²
    """
    np.random.seed(42)
    n = 50  # Number of families
    
    # Generate parent values
    parent1 = np.random.normal(100, 15, n)
    parent2 = np.random.normal(100, 15, n)
    midparent = (parent1 + parent2) / 2
    
    # Generate offspring with heritability
    # Offspring = h² × midparent + (1-h²) × population_mean + environmental noise
    genetic_component = h2_true * midparent + (1 - h2_true) * 100
    environmental_noise = np.random.normal(0, 10 * np.sqrt(1 - h2_true), n)
    offspring = genetic_component + environmental_noise
    
    # Linear regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(midparent, offspring)
    h2_estimated = slope
    
    # Visualization
    fig = go.Figure()
    
    # Scatter plot
    fig.add_trace(go.Scatter(
        x=midparent, y=offspring,
        mode='markers',
        marker=dict(size=8, color='#3498DB'),
        name='Families'
    ))
    
    # Regression line
    x_line = np.array([min(midparent), max(midparent)])
    y_line = slope * x_line + intercept
    fig.add_trace(go.Scatter(
        x=x_line, y=y_line,
        mode='lines',
        line=dict(color='#E74C3C', width=3),
        name=f'Regression (slope={slope:.3f})'
    ))
    
    # Identity line (perfect heritability)
    fig.add_trace(go.Scatter(
        x=x_line, y=x_line,
        mode='lines',
        line=dict(color='gray', width=2, dash='dash'),
        name='Perfect heritability (h²=1)'
    ))
    
    fig.update_layout(
        title=f'<b>Parent-Offspring Regression (True h²={h2_true:.2f})</b>',
        xaxis_title='Mid-parent Value',
        yaxis_title='Offspring Value',
        height=500
    )
    
    # Results
    print("\n" + "="*70)
    print("PARENT-OFFSPRING REGRESSION")
    print("="*70)
    print(f"\nDATA: {n} families")
    print(f"\nTRUE HERITABILITY: h² = {h2_true:.3f}")
    print(f"\nREGRESSION RESULTS:")
    print(f"  Slope (h² estimate):    {h2_estimated:.3f}")
    print(f"  Intercept:              {intercept:.2f}")
    print(f"  R²:                     {r_value**2:.3f}")
    print(f"  P-value:                {p_value:.6f}")
    print(f"  Standard error:         {std_err:.3f}")
    print(f"\nESTIMATED h²: {h2_estimated:.3f}")
    print(f"Error: {abs(h2_estimated - h2_true):.3f}")
    print(f"\nINTERPRETATION:")
    print(f"  Slope = h² because:")
    print(f"  • Offspring inherit genes from both parents")
    print(f"  • Mid-parent value = average genetic contribution")
    print(f"  • Slope measures how much offspring resemble parents")
    print(f"  • If h²=1: offspring = parents (slope=1)")
    print(f"  • If h²=0: offspring = population mean (slope=0)")
    print("="*70)
    
    fig.show()

# Interactive
h2_true_slider = FloatSlider(value=0.6, min=0, max=1, step=0.05,
                            description='True h²:')

display(HTML("<h3>📊 Parent-Offspring Regression</h3>"))
interact(parent_offspring_regression, h2_true=h2_true_slider);

## Part 6: Challenge Problems

### Challenge 1: Calculate h² from Variance 📐

**Given:**
- Plant height variance in identical environment: V = 25 cm²
- Plant height variance in variable environment: V = 100 cm²

**Questions:**
1. Calculate V_G and V_E
2. Calculate heritability (h²)
3. Interpret the result

<details>
<summary>Solution</summary>

**1. Identify Variance Components:**

**In identical environment:**
- No environmental variation → V = V_G only
- Therefore: **V_G = 25 cm²**

**In variable environment:**
- Both genetic and environmental → V = V_G + V_E
- 100 = V_G + V_E
- 100 = 25 + V_E
- Therefore: **V_E = 75 cm²**

**2. Calculate h²:**

Formula: h² = V_G / V_P

V_P = V_G + V_E = 25 + 75 = 100 cm²

h² = 25 / 100 = **0.25**

**3. Interpretation:**

**h² = 0.25 (25%) = LOW heritability**

**What this means:**
- Only 25% of height variation is genetic
- 75% is environmental (soil, water, sunlight)
- Selection will be SLOW
- Environment matters MORE than genes!

**For breeding:**
- Hard to improve by selection alone
- Must control environment carefully
- Response to selection: R = 0.25 × S

**Example:** If you select parents 10 cm taller:
R = 0.25 × 10 = 2.5 cm increase in offspring

Only 25% of the parental advantage is inherited!
</details>

### Challenge 2: Breeder's Equation Application 🌾

**Scenario:** Corn yield breeding
- Population mean: 100 bushels/acre
- Selected parents mean: 120 bushels/acre
- Heritability: h² = 0.60

**Questions:**
1. Calculate selection differential (S)
2. Predict offspring yield using breeder's equation
3. How many generations to reach 130 bushels/acre?

<details>
<summary>Solution</summary>

**1. Selection Differential:**

S = Selected mean - Population mean
S = 120 - 100 = **20 bushels/acre**

**2. Offspring Yield:**

**Breeder's Equation: R = h² × S**

R = 0.60 × 20 = **12 bushels/acre**

Offspring mean = 100 + 12 = **112 bushels/acre**

**Not 120!** Only 60% of parental advantage is inherited.

**3. Generations to 130:**

**Method 1: Iterative**

Start: 100
Gen 1: 100 + 12 = 112 (if we keep S=20)
Gen 2: 112 + 12 = 124
Gen 3: 124 + 12 = 136 → **Exceeds target!**

**Answer: 3 generations** (assuming constant S and h²)

**Method 2: Formula**

Total change needed: 130 - 100 = 30 bushels
Change per generation: R = 12 bushels
Generations = 30 / 12 = 2.5 → **3 generations**

**Important caveats:**
- Assumes h² stays constant (often decreases)
- Assumes same S each generation (gets harder)
- Assumes no environmental changes
- Real breeding takes longer!

**Why breeding is hard:**
- h² decreases as favorable alleles fix
- S decreases as variation depletes
- Hit genetic limits (all favorable alleles fixed)
- Takes many generations in reality
</details>

### Challenge 3: Interpret Heritability 🤔

**Human IQ has h² ≈ 0.50**

**Questions:**
1. Does this mean IQ is 50% genetic?
2. Can we change IQ?
3. Why is h² population-specific?

<details>
<summary>Solution</summary>

**1. "50% genetic"?**

**YES and NO - needs careful interpretation!**

**What h²=0.5 DOES mean:**
- 50% of IQ VARIATION in population is due to genetic variation
- In that specific population
- In that specific environment
- At that specific time

**What h²=0.5 DOES NOT mean:**
- ❌ NOT: "Your IQ is 50% genes, 50% environment"
- ❌ NOT: "Genes determine 50% of IQ"
- ❌ NOT: "Half your IQ from genes, half from learning"

**Correct interpretation:**
"Of the DIFFERENCES between people, 50% are due to genetic differences and 50% to environmental differences"

**Analogy:**
If everyone had identical genes → h²=0 (all variation environmental)
If everyone had identical environment → h²=1 (all variation genetic)

**2. Can we change IQ?**

**YES! Absolutely!**

**h² does NOT mean unchangeable!**

**Evidence:**
- **Flynn Effect:** Average IQ increased 30 points in 100 years
- Genes didn't change → Environment changed!
- Better nutrition, education, healthcare

**Height analogy:**
- Height h² ≈ 0.80 (very heritable)
- BUT average height increased 10 cm in 100 years
- Better nutrition → everyone taller

**Key insight:**
High heritability ≠ fixed trait
Change environment → change trait!

**3. Why population-specific?**

**h² depends on BOTH genetic AND environmental variation**

**Example: Plant height**

**Population A (greenhouse):**
- Uniform environment → low V_E
- Genetic differences visible → high V_G
- Result: h² high

**Population B (wild):**
- Variable environment → high V_E
- Same genetic variation → same V_G
- Result: h² low

**Same genes, different heritability!**

**Why this matters:**
- h² in rich country ≠ h² in poor country
- h² in past ≠ h² in present
- Cannot compare h² across populations
- Cannot use h² for policy conclusions

**Critical point:**
High heritability within groups does NOT imply genetic differences between groups!

**Example:**
- Corn planted in good vs poor soil
- High h² within each field
- Difference between fields is 100% environmental
</details>

In [ ]:
def export_results():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Export heritability database
    data = []
    for trait, info in trait_heritabilities.items():
        data.append({
            'Trait': trait,
            'Heritability_h2': info['h2'],
            'Category': info['category'],
            'Unit': info['unit']
        })
    df = pd.DataFrame(data)
    csv_file = f"/content/lab_2_2_heritabilities_{timestamp}.csv"
    df.to_csv(csv_file, index=False)
    print(f"✓ Saved: {csv_file}")
    print(f"Exported {len(data)} trait heritabilities")

btn = Button(description='📥 Export', button_style='success', icon='download')
btn.on_click(lambda b: export_results())
display(HTML("<h3>📤 Export</h3>"))
display(btn)

## Summary

### Key Concepts

✅ **Heritability (h²):** Proportion of variation due to genes  
✅ **Breeder's Equation:** R = h² × S predicts selection response  
✅ **Variance Partitioning:** V_P = V_G + V_E  
✅ **Parent-Offspring:** Regression slope estimates h²  
✅ **Population-Specific:** h² varies with environment  

### Key Equations

**Heritability:**
## h² = V_G / V_P = V_G / (V_G + V_E)

**Breeder's Equation:**
## R = h² × S

Where:
- R = Response (change in offspring)
- h² = Narrow-sense heritability
- S = Selection differential

**Realized Heritability:**
## h² = R / S

(Estimated from actual breeding experiment)

### Heritability Ranges

**Very High (0.8-1.0):**
- Flower color (0.95)
- Dog size (0.90)
- Butterfly wing pattern (0.85)

**High (0.6-0.8):**
- Human height (0.80)
- Bird beak depth (0.79)
- Corn yield (0.70)

**Moderate (0.4-0.6):**
- Human IQ (0.50)
- Human BMI (0.47)

**Low (0.2-0.4):**
- Cow milk yield (0.35)
- Type 2 diabetes (0.26)
- Chicken egg count (0.20)

### Critical Insights

**1. High h² ≠ Unchangeable**
- Height: h²=0.80, but increased 10cm in century
- Change environment → change trait
- Flynn Effect: IQ up 30 points despite h²=0.50

**2. h² is Population-Specific**
- Depends on genetic AND environmental variation
- Different populations → different h²
- Cannot compare across groups

**3. High h² Within ≠ Differences Between**
- High heritability within groups
- Does NOT imply genetic differences between groups
- Group differences can be 100% environmental

**4. Selection Requires Heritable Variation**
- h²=0 → No response to selection
- h²=1 → Maximum response
- Real breeding: h² usually 0.2-0.8

### Breeding Applications

**Successful:**
- Corn yield: 6× increase in 70 years
- Dog breeds: Extreme morphology in 150 years
- Dairy cows: Doubled milk production

**Why effective:**
- High heritabilities for selected traits
- Strong selection (extreme parents)
- Many generations
- Controlled environments

### The Big Picture

**Variation is the raw material of evolution**

Without variation → No evolution possible
With heritable variation → Selection can act

**Heritability determines:**
- How fast populations can evolve
- How effective artificial selection is
- Whether traits can respond to environment

**Evolution requires:**
1. Variation (differences exist)
2. Heritability (offspring resemble parents)
3. Selection (some variants reproduce more)

**All three present → Evolution inevitable!**

### Next Lab

**Lab 2.3: Extinction Event Analyzer** - When evolution fails!

**Congratulations!** 🎉